In [ ]:
import sys
import os
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import torch
from tqdm import tqdm
import numpy as np

os.environ['HF_HOME'] = '/scratch/' + str(open('../tokens/HPC_ACCOUNT_ID.txt', 'r').read())
cache_dir = '/scratch/' + str(open('../tokens/HPC_ACCOUNT_ID.txt', 'r').read()) + '/cache'

In [ ]:
hf_api_key = ""
with open("../tokens/HF_TOKEN.txt", "r") as f:
    hf_api_key = f.read().strip()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def classify_docs_per_distractor(row, model, tokenizer, device):
    """
    Classify documents for a given distractor.
    """
    

In [ ]:
data = load_dataset("LeoZotos/" + "immu" + "_full", split='train', token = hf_api_key, cache_dir=cache_dir)

correct_answer_docs = []
distractor_1_docs = []
distractor_2_docs = []
distractor_3_docs = []
    

question_set.add_column("relevant_docs" + "_" + 'simple', relevant_docs)
